## Multitemporal RGB composite 2017

### Import the Python packages

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob
os.environ['_CIOP_APPLICATION_PATH']=''
sys.path.append('/opt/anaconda/bin/')
import cioppy
ciop = cioppy.Cioppy()

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc

from shapely.wkt import loads 
from shapely.geometry import box
from shapely.geometry import Point
from shapely import geometry

import osr
import ogr

import gdal
from geopandas import GeoDataFrame
import pandas as pd

sys.path.append(os.getcwd())
import ellip_snap_helpers

from PIL import Image

%load_ext autoreload
%autoreload 2

### Search parameters

Set the catalogue endpoint to Sentinel-1:

In [2]:
series = 'https://catalog.terradue.com/sentinel1/search'

Define the time of interest:

In [19]:
tois = {'winter': { 'start_date': '2017-01-01T00:00:00', 'stop_date': '2017-01-31T23:59:59' },
        'mid_summer': { 'start_date': '2017-07-01T00:00:00', 'stop_date': '2017-07-31T23:59:59' },
        'late_summer': { 'start_date': '2017-08-15T00:00:00', 'stop_date': '2017-08-31T23:59:59' }}

In [4]:
for index, toi in enumerate(tois):
    
    print toi, tois[toi]['start_date'], tois[toi]['stop_date']

mid_summer 2016-07-01T00:00:00 2016-07-31T23:59:59
late_summer 2016-08-15T00:00:00 2016-08-31T23:59:59
winter 2016-01-01T00:00:00 2016-01-31T23:59:59


Define the area of interest:

In [5]:
def extend_aoi(center_x, center_y, extent):
    
    polar_epsg = 3575 # 3995
    latlon_epsg = 4326
    
    center_polar = loads(convert_coords(latlon_epsg, polar_epsg, Point(center_x, center_y).wkt))
    
    ll = convert_coords(polar_epsg, latlon_epsg, Point(center_polar.x - extent,  center_polar.y - extent).wkt)
    lr = convert_coords(polar_epsg, latlon_epsg, Point(center_polar.x + extent,  center_polar.y - extent).wkt)
    ur = convert_coords(polar_epsg, latlon_epsg, Point(center_polar.x + extent,  center_polar.y + extent).wkt)
    ul = convert_coords(polar_epsg, latlon_epsg, Point(center_polar.x - extent,  center_polar.y + extent).wkt)


    pointList = [loads(ll),
             loads(lr), 
             loads(ur), 
             loads(ul), 
             loads(ll)]

    extended_aoi = geometry.Polygon([[p.x, p.y] for p in pointList]).wkt
    
    return extended_aoi

In [6]:
def convert_coords(source_epsg, target_epsg, geom):

    source = osr.SpatialReference()
    source.ImportFromEPSG(source_epsg)

    target = osr.SpatialReference()
    target.ImportFromEPSG(target_epsg)

    transform = osr.CoordinateTransformation(source, target)

    point = ogr.CreateGeometryFromWkt(geom)
    point.Transform(transform)

    return point.ExportToWkt()

In [7]:
poi = loads('POINT (-35.3 83.90000000000001)')

In [8]:
extended_aoi = extend_aoi(poi.x, poi.y, 50000)

extended_aoi

'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.53393836008161, -41.2248291287458 83.8638684209569, -35.2717796499525 83.2658579085283))'

### Build and submit the catalog search


In [9]:
results = []

for index, toi in enumerate(tois):
    
    print toi
    
    search_params = dict([('geom', extended_aoi),
                          ('start', tois[toi]['start_date']),
                          ('stop', tois[toi]['stop_date']),
                          ('track', '117'),
                          ('pt', 'GRD'),
                          ('psn', 'S1A'),
                          ('count', 1)])
    
    print search_params

    search = ciop.search(end_point = series,
                     params = search_params,
                     output_fields='self,enclosure,identifier,startdate,wkt', 
                     model='GeoTime')
    
    print search[0]['identifier']    
    results.append(search[0])

results = GeoDataFrame(results)

mid_summer
{'count': 1, 'pt': 'GRD', 'track': '99', 'stop': '2016-07-31T23:59:59', 'start': '2016-07-01T00:00:00', 'geom': 'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.53393836008161, -41.2248291287458 83.8638684209569, -35.2717796499525 83.2658579085283))'}
S1A_EW_GRDM_1SSH_20160721T114238_20160721T114338_012246_013063_933F
late_summer
{'count': 1, 'pt': 'GRD', 'track': '99', 'stop': '2016-08-31T23:59:59', 'start': '2016-08-15T00:00:00', 'geom': 'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.53393836008161, -41.2248291287458 83.8638684209569, -35.2717796499525 83.2658579085283))'}
S1A_EW_GRDM_1SSH_20160826T114240_20160826T114340_012771_0141E5_BA20
winter
{'count': 1, 'pt': 'GRD', 'track': '99', 'stop': '2016-01-31T23:59:59', 'start': '2016-01-01T00:00:00', 'geom': 'POLYGON ((-35.2717796499525 83.2658579085283, -29.3687708669507 83.8704727024008, -35.334759858866 84.5339383

In [10]:
results

,enclosure,identifier,self,startdate,wkt
0,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160721T114238_20160721T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-07-21T11:42:38.3667520Z,"POLYGON((-46.314613 80.456032,-67.940353 82.33..."
1,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160826T114240_20160826T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-08-26T11:42:40.2897880Z,"POLYGON((-46.307064 80.454826,-67.930687 82.33..."
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20160123T114236_20160123T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-01-23T11:42:36.0966700Z,"POLYGON((-47.054169 80.258202,-68.551834 82.12..."


In [11]:
results['startdate'] = pd.to_datetime(results['startdate']) 
results['wkt'] = results['wkt'].apply(loads)

In [12]:
results

,enclosure,identifier,self,startdate,wkt
0,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160721T114238_20160721T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-07-21 11:42:38.366752,"POLYGON ((-46.314613 80.45603199999999, -67.94..."
1,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160826T114240_20160826T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-08-26 11:42:40.289788,"POLYGON ((-46.307064 80.454826, -67.9306870000..."
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20160123T114236_20160123T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-01-23 11:42:36.096670,"POLYGON ((-47.054169 80.258202, -68.551834 82...."


In [13]:
def analyse(row, aoi_wkt):

    aoi = loads(aoi_wkt)

    aoi_intersection = (row['wkt'].intersection(aoi).area / aoi.area) * 100
        
    series = dict([('aoi_intersection', aoi_intersection)])
    
    return pd.Series(series)

In [14]:
results = results.merge(results.apply(lambda row: analyse(row, extended_aoi), axis=1), 
              left_index=True,
              right_index=True)

In [15]:
results

,enclosure,identifier,self,startdate,wkt,aoi_intersection
0,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160721T114238_20160721T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-07-21 11:42:38.366752,"POLYGON ((-46.314613 80.45603199999999, -67.94...",36.810757
1,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SSH_20160826T114240_20160826T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-08-26 11:42:40.289788,"POLYGON ((-46.307064 80.454826, -67.9306870000...",36.788618
2,https://store.terradue.com/download/sentinel1/...,S1A_EW_GRDM_1SDH_20160123T114236_20160123T1143...,https://catalog.terradue.com/sentinel1/search?...,2016-01-23 11:42:36.096670,"POLYGON ((-47.054169 80.258202, -68.551834 82....",11.612867


In [16]:
def stage_in(row):
    
    local_path = ciop.copy(row['enclosure'], extract=False, target='/data2')
    row['local_path'] = local_path
        
    return row 
    

In [18]:
results = results.apply(lambda row: stage_in(row), axis=1)

Exception: ("subprocess returned: 1\n2018-11-19T12:16:44 [INFO   ][ciop-copy][starting] url 'https://store.terradue.com/download/sentinel1/files/v1/S1A_EW_GRDM_1SSH_20160721T114238_20160721T114338_012246_013063_933F' > local '/data2/'\n2018-11-19T12:33:02 [WARN  ][ciop-copy][failed] url 'https://store.terradue.com/download/sentinel1/files/v1/S1A_EW_GRDM_1SSH_20160721T114238_20160721T114338_012246_013063_933F' not found\n2018-11-19T12:33:02 [INFO  ][ciop-copy][info] attempting https://store.terradue.com/download/sentinel1/files/v1/S1A_EW_GRDM_1SSH_20160721T114238_20160721T114338_012246_013063_933F.gz\n2018-11-19T12:33:02 [INFO   ][ciop-copy][starting] url 'https://store.terradue.com/download/sentinel1/files/v1/S1A_EW_GRDM_1SSH_20160721T114238_20160721T114338_012246_013063_933F.gz' > local '/data2/'\nbasename: missing operand\nTry `basename --help' for more information.\n2018-11-19T12:33:02 [ERROR  ][ciop-copy][failed] url 'https://store.terradue.com/download/sentinel1/files/v1/S1A_EW_GR

In [ ]:
results

### Process sigma0

In [ ]:
def sigma0(row, aoi):
    
    local_path = row['local_path']
    identifier = row['identifier']
    
    mygraph = ellip_snap_helpers.GraphProcessor()
    
    operator = 'Read'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['file'] = local_path 
    mygraph.add_node(operator,
                     operator, 
                     parameters,
                     '') 
    
    operator = 'ThermalNoiseRemoval'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Read')
    

    operator = 'Apply-Orbit-File'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator, 
                     operator, 
                     parameters, 
                     'ThermalNoiseRemoval')



    operator = 'Calibration'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters, 
                     'Apply-Orbit-File')


    operator = 'Speckle-Filter'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator, 
                     operator,
                     parameters, 
                     'Calibration')


    operator = 'Multilook'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Speckle-Filter')


    operator = 'LinearToFromdB'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Multilook')


    operator = 'Terrain-Correction'

    map_proj = """PROJCS["WGS 84 / North Pole LAEA Europe",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Lambert_Azimuthal_Equal_Area"],
    PARAMETER["latitude_of_center",90],
    PARAMETER["longitude_of_center",10],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    AUTHORITY["EPSG","3575"]]"""

    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['demName'] = 'ACE30'  
    parameters['saveDEM'] = 'true'
    parameters['mapProjection'] = map_proj
    parameters['nodataValueAtSea'] = 'false'   
                

    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'LinearToFromdB')

    operator = 'Subset'
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['geoRegion'] = aoi 

    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Terrain-Correction')


    operator = 'Write'
    
    output_name = 'SIGMA0_%s' % identifier
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['file'] = output_name
    parameters['formatName'] = 'GeoTIFF-BigTiff'
    mygraph.add_node(operator,
                     operator,
                     parameters,
                     'Subset')

    mygraph.run()
    
    row['sigma0'] = output_name + '.tif'
    
    return row

In [ ]:
results = results.apply(lambda row: sigma0(row, extended_aoi), axis=1)

In [ ]:
results

In [ ]:
sigma0s = list(results['sigma0'].values)

In [ ]:
sigma0s

In [ ]:
ds = gdal.Open(sigma0s[0])

geo_transform = ds.GetGeoTransform()
proj = ds.GetProjection()

srs=osr.SpatialReference(wkt=proj)

w = ds.GetRasterBand(1).XSize
h = ds.GetRasterBand(1).YSize

ds = None

In [ ]:
bands = []

min_db = -26
max_db = 6

rgb_name = 'multitemporal_rgb' + '.tif'
    
options = ['PHOTOMETRIC=RGB', 'PROFILE=GeoTIFF']
    
dst_ds = gdal.GetDriverByName('GTiff').Create(rgb_name, w, h, 3, gdal.GDT_Byte, options=options)
    
dst_ds.SetGeoTransform(geo_transform)   
 
dst_ds.SetProjection(srs.ExportToWkt()) 

for index, tif in enumerate(sigma0s):
    
    raster = gdal.Open(tif)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray(0, 0, w, h)
    
    band_dest = (array * 255 / (max_db - min_db))
    
    bands.append(band_dest)
    
    dst_ds.GetRasterBand(index + 1).WriteArray(band_dest.astype(np.uint8))
    
dst_ds.FlushCache()                    
dst_ds = None

In [ ]:
rgb_uint8 = np.dstack(bands).astype(np.uint8) 

width = 20
height = 20
plt.figure(figsize=(width, height))
img = Image.fromarray(rgb_uint8)
imgplot = plt.imshow(img)